### Import libraries

In [1]:
## this code is developed by Jose Maria Alonso-Moral

import os

# Setting Java8 is needed for compatibility with JFML (IEEE standard 1855-2016)
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
!java -version
os.system("pip install simplenlg --quiet")

java version "17.0.4.1" 2022-08-18 LTS
Java(TM) SE Runtime Environment (build 17.0.4.1+1-LTS-2)
Java HotSpot(TM) 64-Bit Server VM (build 17.0.4.1+1-LTS-2, mixed mode, sharing)


0

In [2]:
from simplenlg.framework import *
from simplenlg.lexicon import *
from simplenlg.realiser.english import *
from simplenlg.phrasespec import *
from simplenlg.features import *

import pandas as pd
import numpy as np
from ast import literal_eval
from sklearn.neighbors import NearestNeighbors

### Functions

In [3]:
def first_phrase_explanation(movie, recommendation, genres):
    lexicon = Lexicon.getDefaultLexicon()
    nlgFactory = NLGFactory(lexicon)
    realiser = Realiser(lexicon)

     # SENTENCE 1
    # PART 1
    sentence1_1 = nlgFactory.createClause()
    subj = nlgFactory.createNounPhrase("we")
    sentence1_1.setSubject(subj)
    sentence1_1.setVerb("recommend")
    sentence1_1.setIndirectObject("you")
    sentence1_1.setObject(recommendation)

    sentence1_1.setFeature(Feature.TENSE, Tense.PRESENT)

    # PART 2
    sentence1_2 = nlgFactory.createClause("it", "be", "the most similar")
    similar_clause = nlgFactory.createClause(movie)
    similar_clause.setFeature(Feature.COMPLEMENTISER, "to")
    sentence1_2.addComplement(similar_clause)

    # PART 3
    sentence1_3 = nlgFactory.createClause()
    subj = nlgFactory.createNounPhrase("genre list")
    subj.setDeterminer("its")
    sentence1_3.setSubject(subj)
    sentence1_3.setVerb("be")
    sentence1_3.setFeature(Feature.TENSE, Tense.PRESENT);

    # PART 4
    if len(genres) > 0:
        genre1 = nlgFactory.createNounPhrase(genres[0]);
        sentence1_4 = nlgFactory.createCoordinatedPhrase(genre1);
        for i in range(1, len(genres)):
            sentence1_4.addCoordinate(genres[i]);
        
    sentence1_4.setFeature(Feature.COMPLEMENTISER, "")
    sentence1_3.addComplement(sentence1_4)

    # UNION
    sentence1_2.setFeature(Feature.COMPLEMENTISER, "because")
    sentence1_1.addComplement(sentence1_2)
    sentence_1 = nlgFactory.createCoordinatedPhrase(sentence1_1, sentence1_3)
    print(realiser.realiseSentence(sentence_1))

In [4]:
def second_phrase_explanation(recommendation, previous_recommendation, popularity):
    lexicon = Lexicon.getDefaultLexicon()
    nlgFactory = NLGFactory(lexicon)
    realiser = Realiser(lexicon)

     # SENTENCE 2
    # PART 1
    sentence2_1 = nlgFactory.createClause()
    subj = nlgFactory.createNounPhrase("you")
    sentence2_1.setSubject(subj)
    sentence2_1.addModifier("also")
    sentence2_1.setVerb("may like")
    sentence2_1.setObject(recommendation)

    # PART 2
    sentence2_2 = nlgFactory.createClause("it", "be")
    sentence2_2.addComplement("more")
    sentence2_2.addComplement("popular")
    sentence2_2.addComplement("than")
    sentence2_2.addComplement(previous_recommendation)
    
    # UNION
    sentence2_2.setFeature(Feature.COMPLEMENTISER, "because")
    sentence2_1.addComplement(sentence2_2)
    sentence2_1.addComplement("with")
    sentence2_1.addComplement("an")
    sentence2_1.addComplement("average")
    sentence2_1.addComplement("of")
    sentence2_1.addModifier(str(popularity))
    
    print(realiser.realiseSentence(sentence2_1))

In [5]:
def third_phrase_explanation(movie, recommendation_list):
    lexicon = Lexicon.getDefaultLexicon()
    nlgFactory = NLGFactory(lexicon)
    realiser = Realiser(lexicon)

    # SENTENCE 3
    # PART 1
    sentence3_1 = nlgFactory.createClause()
    sentence3_1.setObject("some other films")
    subj = nlgFactory.createNounPhrase("you")
    sentence3_1.setSubject(subj)
    sentence3_1.setVerb("may like")
    
    # PART 2
    sentence3_2 = nlgFactory.createClause()
    subj = nlgFactory.createNounPhrase("you")
    sentence3_2.setSubject(subj)
    sentence3_2.setVerb("enjoy")
    sentence3_2.setFeature(Feature.TENSE, Tense.PAST);
    sentence3_2.setObject(movie)

    # UNION
    sentence3_2.setFeature(Feature.COMPLEMENTISER, "if")
    sentence3_1.addComplement(sentence3_2)
    print(realiser.realiseSentence(sentence3_1))

    # PART 3
    for movie, similarity_level in recommendation_list:

        # PART 3.1
        sentence1 = nlgFactory.createClause()
        subj = nlgFactory.createNounPhrase(f"\"{movie}\"")
        sentence1.setSubject(subj)

        # PART 3.2
        sentence2 = nlgFactory.createClause("it", "be", f"{calculate_similarity(similarity_level)}")

        # UNION
        sentence2.setFeature(Feature.COMPLEMENTISER, "because")
        sentence1.addComplement(sentence2)

        print(realiser.realiseSentence(sentence1))

In [6]:
def calculate_similarity(distance):
    if distance >= 0.85:
        return "\"Something new\""
    elif distance >= 0.7:
        return "\"Slightly Similar\""
    elif distance >= 0.4:
        return "\"Similar\""
    else:
        return "\"Very Similar\""

In [7]:
def make_explanation(movie, recommendation_1, recommendation_2, recommendation_list):
    """
    movie : str
        The name of the movie used to make the recomendations
    recommendation_1 : dict
        The closest movie in terms of distance
    recommendation_2 : dict
        The movie with the highest popularity among the recommended
    recommendation_list : list
        A list containing (name of the recommended movie, distance to the base movie)
    """

    # Extracting the data
    name_recommendation_1 = recommendation_1['title']
    genres_recommendation_1 = recommendation_1['genres']
    popularity_recommendation_1 = recommendation_1['popularity']

    name_recommendation_2 = recommendation_2['title']
    genres_recommendation_2 = recommendation_2['genres']
    popularity_recommendation_2 = recommendation_2['popularity']

    # Building the explanations
    # PHRASE 1
    first_phrase_explanation(
        movie=movie,
        recommendation=name_recommendation_1,
        genres = genres_recommendation_1
    )

    # PHRASE 2
    second_phrase_explanation(
        recommendation=name_recommendation_2,
        previous_recommendation=name_recommendation_1,
        popularity=popularity_recommendation_2
    )
    print("\n")
    
    # PHRASE 3
    third_phrase_explanation(
        movie=movie,
        recommendation_list=recommendation_list
    )

### Load & Preprocess data

In [9]:
tmdb_5000_credits = pd.read_csv("datasets/tmdb_5000_credits.csv")
tmdb_5000_movies = pd.read_csv("datasets/tmdb_5000_movies.csv")

In [10]:
columns_movies = ["budget", "genres", "id", "original_language", "original_title", "overview", "popularity", "production_countries", "release_date", "spoken_languages", "vote_average", "vote_count"]
columns_credits = ["movie_id", "title", "cast", "crew"]

tmdb_5000_movies = tmdb_5000_movies[columns_movies]
tmdb_5000_credits = tmdb_5000_credits[columns_credits]

for column in ["genres", "production_countries", "spoken_languages"]:
    tmdb_5000_movies[column] = tmdb_5000_movies[column].apply(literal_eval)

for column in ["crew", "cast"]:
    tmdb_5000_credits[column] = tmdb_5000_credits[column].apply(literal_eval)    

In [11]:
tmdb_5000_credits.head(5)

,movie_id,title,cast,crew
0,19995,Avatar,"[{'cast_id': 242, 'character': 'Jake Sully', '...","[{'credit_id': '52fe48009251416c750aca23', 'de..."
1,285,Pirates of the Caribbean: At World's End,"[{'cast_id': 4, 'character': 'Captain Jack Spa...","[{'credit_id': '52fe4232c3a36847f800b579', 'de..."
2,206647,Spectre,"[{'cast_id': 1, 'character': 'James Bond', 'cr...","[{'credit_id': '54805967c3a36829b5002c41', 'de..."
3,49026,The Dark Knight Rises,"[{'cast_id': 2, 'character': 'Bruce Wayne / Ba...","[{'credit_id': '52fe4781c3a36847f81398c3', 'de..."
4,49529,John Carter,"[{'cast_id': 5, 'character': 'John Carter', 'c...","[{'credit_id': '52fe479ac3a36847f813eaa3', 'de..."


In [12]:
tmdb_5000_movies.head(5)

,budget,genres,id,original_language,original_title,overview,popularity,production_countries,release_date,spoken_languages,vote_average,vote_count
0,237000000,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",19995,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{'iso_3166_1': 'US', 'name': 'United States o...",2009-12-10,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",7.2,11800
1,300000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",285,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{'iso_3166_1': 'US', 'name': 'United States o...",2007-05-19,"[{'iso_639_1': 'en', 'name': 'English'}]",6.9,4500
2,245000000,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",206647,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{'iso_3166_1': 'GB', 'name': 'United Kingdom'...",2015-10-26,"[{'iso_639_1': 'fr', 'name': 'Français'}, {'is...",6.3,4466
3,250000000,"[{'id': 28, 'name': 'Action'}, {'id': 80, 'nam...",49026,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{'iso_3166_1': 'US', 'name': 'United States o...",2012-07-16,"[{'iso_639_1': 'en', 'name': 'English'}]",7.6,9106
4,260000000,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",49529,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{'iso_3166_1': 'US', 'name': 'United States o...",2012-03-07,"[{'iso_639_1': 'en', 'name': 'English'}]",6.1,2124


In [13]:
def preprocess_dict(column: str, key: str = "name"):
    genres_column = tmdb_5000_movies[column]
    genres_column_processed = []
    for item in genres_column:
        genres_column_processed.append([dic[key] for dic in item])
    return genres_column_processed


In [14]:
tmdb_5000_movies["genres"] = preprocess_dict("genres")
tmdb_5000_movies["production_countries"] = preprocess_dict("production_countries")
tmdb_5000_movies["spoken_languages"] = preprocess_dict("spoken_languages")

In [15]:
tmdb_5000_movies.head(5)

,budget,genres,id,original_language,original_title,overview,popularity,production_countries,release_date,spoken_languages,vote_average,vote_count
0,237000000,"[Action, Adventure, Fantasy, Science Fiction]",19995,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[United States of America, United Kingdom]",2009-12-10,"[English, Español]",7.2,11800
1,300000000,"[Adventure, Fantasy, Action]",285,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,[United States of America],2007-05-19,[English],6.9,4500
2,245000000,"[Action, Adventure, Crime]",206647,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[United Kingdom, United States of America]",2015-10-26,"[Français, English, Español, Italiano, Deutsch]",6.3,4466
3,250000000,"[Action, Crime, Drama, Thriller]",49026,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,[United States of America],2012-07-16,[English],7.6,9106
4,260000000,"[Action, Adventure, Science Fiction]",49529,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,[United States of America],2012-03-07,[English],6.1,2124


In [16]:
tmdb_5000_movies = tmdb_5000_movies.merge(tmdb_5000_credits, left_on='id', right_on='movie_id')

In [17]:
print(tmdb_5000_movies.shape)
tmdb_5000_movies = tmdb_5000_movies[tmdb_5000_movies['budget'].apply(lambda x: x > 0)]
tmdb_5000_movies = tmdb_5000_movies[tmdb_5000_movies['genres'].apply(lambda x: len(x) > 0)]
print(tmdb_5000_movies.shape)

(4803, 16)
(3762, 16)


In [18]:
tmdb_5000_movies = tmdb_5000_movies.reset_index(drop=True)

In [19]:
tmdb_5000_movies["cast"] = preprocess_dict("cast", "name")

In [20]:
tmdb_5000_movies.head()

,budget,genres,id,original_language,original_title,overview,popularity,production_countries,release_date,spoken_languages,vote_average,vote_count,movie_id,title,cast,crew
0,237000000,"[Action, Adventure, Fantasy, Science Fiction]",19995,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[United States of America, United Kingdom]",2009-12-10,"[English, Español]",7.2,11800,19995,Avatar,"[Sam Worthington, Zoe Saldana, Sigourney Weave...","[{'credit_id': '52fe48009251416c750aca23', 'de..."
1,300000000,"[Adventure, Fantasy, Action]",285,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,[United States of America],2007-05-19,[English],6.9,4500,285,Pirates of the Caribbean: At World's End,"[Johnny Depp, Orlando Bloom, Keira Knightley, ...","[{'credit_id': '52fe4232c3a36847f800b579', 'de..."
2,245000000,"[Action, Adventure, Crime]",206647,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[United Kingdom, United States of America]",2015-10-26,"[Français, English, Español, Italiano, Deutsch]",6.3,4466,206647,Spectre,"[Daniel Craig, Christoph Waltz, Léa Seydoux, R...","[{'credit_id': '54805967c3a36829b5002c41', 'de..."
3,250000000,"[Action, Crime, Drama, Thriller]",49026,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,[United States of America],2012-07-16,[English],7.6,9106,49026,The Dark Knight Rises,"[Christian Bale, Michael Caine, Gary Oldman, A...","[{'credit_id': '52fe4781c3a36847f81398c3', 'de..."
4,260000000,"[Action, Adventure, Science Fiction]",49529,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,[United States of America],2012-03-07,[English],6.1,2124,49529,John Carter,"[Taylor Kitsch, Lynn Collins, Samantha Morton,...","[{'credit_id': '52fe479ac3a36847f813eaa3', 'de..."


In [21]:
tmdb_5000_movies['overview'].head()

0    In the 22nd century, a paraplegic Marine is di...
1    Captain Barbossa, long believed to be dead, ha...
2    A cryptic message from Bond’s past sends him o...
3    Following the death of District Attorney Harve...
4    John Carter is a war-weary, former military ca...
Name: overview, dtype: object

In [22]:
titles = tmdb_5000_movies['original_title']

In [23]:
def get_director(x):
    for i in x:
        if i['job'] == 'Director':
            return i['name']
    return np.nan

In [24]:
def get_list(x):
    if isinstance(x, list):
        if len(x) > 5:
            x = x[:5]
        return x

    return []

In [25]:
tmdb_5000_movies['director'] = tmdb_5000_movies['crew'].apply(get_director)

features = ['cast', 'genres']
for feature in features:
    tmdb_5000_movies[feature] = tmdb_5000_movies[feature].apply(get_list)

In [26]:
def clean_data(x):
    if isinstance(x, list):
        return [str.lower(i.replace(" ", "")) for i in x]
    else:
        if isinstance(x, str):
            return str.lower(x.replace(" ", ""))
        else:
            return ''

In [27]:
features = ['cast', 'director', 'genres']

for feature in features:
    tmdb_5000_movies[feature] = tmdb_5000_movies[feature].apply(clean_data)

In [28]:
def create_soup(x):
    return ' '.join(x['cast']) + ' ' + x['director'] + ' '.join(x['genres'])
tmdb_5000_movies['soup'] = tmdb_5000_movies.apply(create_soup, axis=1)

In [29]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler

tfidf = TfidfVectorizer(stop_words='english')

tfidf_matrix = tfidf.fit_transform(tmdb_5000_movies['soup'])
tfidf_matrix_dense = tfidf_matrix.toarray()
scaler = StandardScaler()
tfidf_matrix_standardized = scaler.fit_transform(tfidf_matrix_dense)

print(tfidf_matrix_standardized.shape)

(3762, 10242)


In [30]:
knn = NearestNeighbors(n_neighbors=15, metric='cosine', algorithm='brute')
knn.fit(tfidf_matrix_standardized)

def get_recommendation(title, k=10):
    idx = tmdb_5000_movies[tmdb_5000_movies['original_title'] == title].index[0]
    distances, indices = knn.kneighbors(tfidf_matrix_standardized[idx].reshape(1, -1), n_neighbors=k+1)
    recommendations = []
    for i in range(1, len(indices[0])):
        recommendations.append(tmdb_5000_movies.iloc[indices[0][i]][['original_title', 'genres', 'popularity', 'cast', 'vote_average', 'director', 'overview']])
    
    return recommendations, distances[0][1:]

In [45]:
movie_name = "The Lord of the Rings: The Fellowship of the Ring"
recommendations, distances = get_recommendation(movie_name, k=10)
print(f"Recommendations if you saw '{movie_name}':")
actores = tmdb_5000_movies.query("original_title == @movie_name")['cast']
for i, recommendation in enumerate(recommendations):
    recommendations[i]['same_actors'] = len(set(actores.values[0]) & set(recommendation['cast']))
    recommendations[i]['distances'] = distances[i]
pd.DataFrame(recommendations)

Recommendations if you saw 'The Lord of the Rings: The Fellowship of the Ring':


,original_title,genres,popularity,cast,vote_average,director,overview,same_actors,distances
328,The Lord of the Rings: The Two Towers,"[adventure, fantasy, action]",106.914973,"[elijahwood, ianmckellen, viggomortensen, livt...",8.0,peterjackson,Frodo and Sam are trekking to Mordor to destro...,3,0.260228
327,The Lord of the Rings: The Return of the King,"[adventure, fantasy, action]",123.630332,"[elijahwood, ianmckellen, viggomortensen, livt...",8.1,peterjackson,Aragorn is revealed as the heir to the ancient...,3,0.260228
98,The Hobbit: An Unexpected Journey,"[adventure, fantasy, action]",108.849621,"[ianmckellen, martinfreeman, richardarmitage, ...",7.0,peterjackson,"Bilbo Baggins, a hobbit enjoying his quiet lif...",2,0.580645
22,The Hobbit: The Desolation of Smaug,"[adventure, fantasy]",94.370564,"[martinfreeman, ianmckellen, richardarmitage, ...",7.6,peterjackson,"The Dwarves, Bilbo and Gandalf have successful...",1,0.722795
24,King Kong,"[adventure, drama, action]",61.226010,"[naomiwatts, jackblack, adrienbrody, thomaskre...",6.6,peterjackson,"In 1933 New York, an overly ambitious movie pr...",0,0.741616
145,Troy,"[adventure, drama, war]",66.803149,"[bradpitt, orlandobloom, ericbana, briancox, s...",6.9,wolfgangpetersen,"In year 1250 B.C. during the late Bronze age, ...",2,0.772071
12,Pirates of the Caribbean: Dead Man's Chest,"[adventure, fantasy, action]",145.847379,"[johnnydepp, orlandobloom, keiraknightley, ste...",7.0,goreverbinski,Captain Jack Sparrow works his way out of a bl...,1,0.835390
1,Pirates of the Caribbean: At World's End,"[adventure, fantasy, action]",139.082615,"[johnnydepp, orlandobloom, keiraknightley, ste...",6.9,goreverbinski,"Captain Barbossa, long believed to be dead, ha...",1,0.857100
848,Elizabethtown,"[comedy, drama, romance]",22.894898,"[orlandobloom, kirstendunst, susansarandon, al...",6.1,cameroncrowe,Drew Baylor is fired after causing his shoe co...,1,0.862175
501,X-Men,"[adventure, action, sciencefiction]",4.668910,"[patrickstewart, hughjackman, ianmckellen, hal...",6.8,bryansinger,"Two mutants, Rogue and Wolverine, come to a pr...",1,0.867402


In [39]:
k = tmdb_5000_movies.query("original_title == @movie_name")
k[['original_title', 'genres', 'popularity', 'cast', 'vote_average', 'director', 'overview']]

,original_title,genres,popularity,cast,vote_average,director,overview
145,Troy,"[adventure, drama, war]",66.803149,"[bradpitt, orlandobloom, ericbana, briancox, s...",6.9,wolfgangpetersen,"In year 1250 B.C. during the late Bronze age, ..."


### Explanation in Natural Language

First recommended movie _(most similar one)_

In [46]:
title_recommendation_1 = f"\"{recommendations[0]['original_title']}\""
genres_recommendation_1 = recommendations[0]['genres']
popularity_recommendation_1 = recommendations[0]['vote_average']

Second recommended movie _(most popular recommended movie)_

In [47]:
recommendation_2 = pd.DataFrame(recommendations).sort_values("vote_average", ascending = False).iloc[0]

In [48]:
title_recommendation_2 = f"\"{recommendation_2['original_title']}\""
genres_recommendation_2 = recommendation_2['genres']
popularity_recommendation_2 = recommendation_2['vote_average']

More recommendations based on similarity

In [49]:
recommendation_list = []
for recommendation in recommendations:
    if f"\"{recommendation['original_title']}\"" != title_recommendation_1 and f"\"{recommendation['original_title']}\"" != title_recommendation_2:
        recommendation_list.append(
            (recommendation['original_title'], recommendation['distances'])
        )
recommendation_list

[('The Hobbit: An Unexpected Journey', 0.5806448932152826),
 ('The Hobbit: The Desolation of Smaug', 0.7227945593207432),
 ('King Kong', 0.7416156394088879),
 ('Troy', 0.7720709085663268),
 ("Pirates of the Caribbean: Dead Man's Chest", 0.8353904567404019),
 ("Pirates of the Caribbean: At World's End", 0.8571001704221616),
 ('Elizabethtown', 0.8621753452140627),
 ('X-Men', 0.8674022624077166)]

**Explanation**

In [50]:
make_explanation(
    movie = f"\"{movie_name}\"",
    recommendation_1 = {
        'title': title_recommendation_1,
        'genres': genres_recommendation_1,
        'popularity': popularity_recommendation_1
    },
    recommendation_2 = {
        'title': title_recommendation_2,
        'genres': genres_recommendation_2,
        'popularity': popularity_recommendation_2
    },
    recommendation_list = recommendation_list
)

We recommend you "The Lord of the Rings: The Two Towers" because it is the most similar to "The Lord of the Rings: The Fellowship of the Ring" and its genre list is adventure, fantasy and action.
Also you may like "The Lord of the Rings: The Return of the King" because it is more popular than "The Lord of the Rings: The Two Towers" with an average of 8.1.


You may like some other films if you enjoyed "The Lord of the Rings: The Fellowship of the Ring".
"The Hobbit: An Unexpected Journey" because it is "Similar".
"The Hobbit: The Desolation of Smaug" because it is "Slightly Similar".
"King Kong" because it is "Slightly Similar".
"Troy" because it is "Slightly Similar".
"Pirates of the Caribbean: Dead Man's Chest" because it is "Slightly Similar".
"Pirates of the Caribbean: At World's End" because it is "Something new".
"Elizabethtown" because it is "Something new".
"X-Men" because it is "Something new".
